In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing all the necessary libraries 
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
import keras
from keras.preprocessing.image import load_img,img_to_array

from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sn
import cv2

import glob
from keras.applications import DenseNet169
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
#Mark all the directories 
train_dir = '../input/chest-xray-pneumonia/chest_xray/train'
test_dir = '../input/chest-xray-pneumonia/chest_xray/test'
val_dir = '../input/chest-xray-pneumonia/chest_xray/val'
pneumonia_train_images = glob.glob(train_dir+"/PNEUMONIA/*.jpeg")
normal_train_images = glob.glob(train_dir+"/NORMAL/*.jpeg")

In [ ]:
#Let us try to plot a pie chart for the data set 
plt.figure(figsize=(10, 6))
plt.pie(x=np.array([len(pneumonia_train_images), len(normal_train_images)]), autopct="%.1f%%", explode=[0.2,0], labels=["pneumonia", "normal"], pctdistance=0.5)
plt.title("Count of Pneumonia Vs Normal", fontsize=12);

In [ ]:
#We can see that this is an imbalanced data set hence let us try to resize the image to something scalable
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(pneumonia_train_images[i])
    img = cv2.resize(img, (512,512))
    ax.imshow(img)
    ax.set_title("Pneumonia")
    
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(normal_train_images[i])
    img = cv2.resize(img, (220,220))
    ax.imshow(img)
    ax.set_title("Normal")
fig.tight_layout()    
plt.show()


In [ ]:
#After looking up for various ways to detect pneumonia it seems that when we use canny edge detection the pneumonia diagnoised x-rays do not appear to come as clear 
#as the ones which which aren't affected. Let us try to check with this.
#Canny Edge Detection Method 
fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(pneumonia_train_images[i])
    img = cv2.resize(img, (512,512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.Canny(img, 80, 100)
    ax.imshow(img)
    ax.set_title("Pneumonia")
fig.tight_layout()
plt.show()

fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    img = cv2.imread(normal_train_images[i])
    img = cv2.resize(img, (512,512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.Canny(img, 80, 100)
    ax.imshow(img)
    ax.set_title("Normal")
fig.tight_layout()    
plt.show()

***As we can see that the images of the patients with Pneumonia, the X rays of them are not clear and the spinal cord part is not as visible as the x rays of the patients who aren't diagonised with pneumonia. Canny Edge Method allows us to clearly understand this. We can see that it is difficult to detect edges in the x-rays with pneumonia whereas the normal images are clear with prominent edges***

In [ ]:
#Now let us try to rescale the images by dividing the pixels with 255
#Also to maintain uniform size let us also resize the images to shape(220,220)
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   zoom_range = 0.2)
val_datagen = ImageDataGenerator(rescale = 1./255.,)
test_datagen = ImageDataGenerator(rescale = 1./255.,)


train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='binary', target_size = (220, 220))
validation_generator = val_datagen.flow_from_directory(test_dir, batch_size=20, class_mode = 'binary', target_size=(220, 220))
test_generator = test_datagen.flow_from_directory(val_dir,shuffle=False, batch_size=20, class_mode = 'binary', target_size=(220, 220))

In [ ]:
#Defining input shape which is to be inserted inside the resnet(or any neural network)
input_shape=(220,200,3)

In [ ]:
#ResNet50 model is used here as base model for transfer learning
#ResNet50 Method Algorithm 
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', input_shape=input_shape, include_top=False)

for layer in base_model.layers:
    layer.trainable = False
    
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
'model.compile(optimizer="adam", loss='binary_crossentropy', metrics=["accuracy"])

'callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

'history = model.fit(train_generator, validation_data=validation_generator, steps_per_epoch = 100, epochs=20, callbacks=callback)

In [ ]:
path="../input/chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0001-0001.jpeg"

img = load_img(path,target_size=(220,200))
input_arr=img_to_array(img)/255

plt.imshow(input_arr)
plt.show()

input_arr.shape

input_arr = np.expand_dims(input_arr , axis = 0)

pred = model.predict(input_arr)
pred
y_pred = []
for prob in pred:
    if prob >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
if y_pred[0] == 0:
    print("The Person Doesn't have pneumonia")
else:
    print("The Person has pneumonia")


**Now let us try to create the Confusion Matrix for the test set****

In [ ]:
pred=model.predict(test_generator)
pred

***Since the predictions are in form of probabilities, we convert the probabilities with less than 0.5 to 0 and probabilities more than equal to 0.5 to 1. This would help us in calculation of metrics of the model.***

In [ ]:
y_pred1 = []
for prob in pred:
    if prob >= 0.5:
        y_pred1.append(1)
    else:
        y_pred1.append(0)
        
y_pred1


In [ ]:
y_true = test_generator.classes
y_true

In [ ]:
#Confusion_Matrix and Heat Map
cm = confusion_matrix(y_true, y_pred1)
sn.heatmap(cm, annot=True,cmap="rainbow_r", annot_kws={"size": 16})

In [ ]:
print(classification_report(y_true, y_pred1))